# MNIST Convolutional Neural Network
* MNIST 데이터셋을 이용해 Convolutional Neural Network를 학습시키고 이미지 분류를 풀어보도록 하겠습니다.

In [18]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
print(keras.__version__)

2.1.0
2.2.4-tf


## Hyper Parameters

In [5]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

tf.random.set_seed(777)

## Datasets(Train Data & Test Data)

In [7]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

# shape 확인
print(x_train.shape)
print(y_train.shape)

# 데이터 channel 추가
x_train = np.expand_dims(x_train, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
x_test = np.expand_dims(x_test, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

# 데이터 pixel 값을 0 ~ 1 사이로 조정 (Normalization)
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

# 레이블 -> One-Hot encoding 변환
y_train = to_categorical(y_train, 10) # [N,] -> [N, 10]
y_test = to_categorical(y_test, 10) # [N,] -> [N, 10]

# shape 확인
print(x_train.shape)
print(y_train.shape)

# tf.data.Datasets 
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

print(train_dataset)
print(test_dataset)

(60000, 28, 28)
(60000,)
(60000, 28, 28, 1)
(60000, 10)
<BatchDataset shapes: ((None, 28, 28, 1), (None, 10)), types: (tf.float32, tf.float32)>
<BatchDataset shapes: ((None, 28, 28, 1), (None, 10)), types: (tf.float32, tf.float32)>


## Model Function(Keras Sequential API)

In [19]:
def create_model_sequantial():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(28, 28, 1)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(10))
    return model

In [26]:
model_sequantial = create_model_sequantial()
model_sequantial.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 2048)             

## Model Function(Keras Functional API)

In [21]:
def create_model_functional():
    inputs = keras.Input(shape=(28, 28, 1))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)(inputs)
    pool1 = keras.layers.MaxPool2D(padding='SAME')(conv1)
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)(pool1)
    pool2 = keras.layers.MaxPool2D(padding='SAME')(conv2)
    conv3 = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)(pool2)
    pool3 = keras.layers.MaxPool2D(padding='SAME')(conv3)
    pool3_flat = keras.layers.Flatten()(pool3)
    dense4 = keras.layers.Dense(units=256, activation=tf.nn.relu)(pool3_flat)
    drop4 = keras.layers.Dropout(rate=0.4)(dense4)
    logits = keras.layers.Dense(units=10)(drop4)
    return keras.Model(inputs=inputs, outputs=logits)

In [27]:
model_functional = create_model_functional()
model_functional.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 4, 4, 128)         0   

## Model Class(Keras Subclassing)

In [30]:
class MNISTModel(tf.keras.Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv1 = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
        self.pool1 = keras.layers.MaxPool2D(padding='SAME')
        self.conv2 = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
        self.pool2 = keras.layers.MaxPool2D(padding='SAME')
        self.conv3 = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
        self.pool3 = keras.layers.MaxPool2D(padding='SAME')
        self.pool3_flat = keras.layers.Flatten()
        self.dense4 = keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.drop4 = keras.layers.Dropout(rate=0.4)
        self.dense5 = keras.layers.Dense(units=10)
    def call(self, inputs, training=False):
        net = self.conv1(inputs)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.pool3(net)
        net = self.pool3_flat(net)
        net = self.dense4(net)
        net = self.drop4(net)
        net = self.dense5(net)
        return net

In [31]:
model = MNISTModel()
temp_inputs = keras.Input(shape=(28, 28, 1))
model(temp_inputs)
model.summary()

Model: "mnist_model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 2048)            

## Loss Function

In [33]:
@tf.function
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(
        y_pred=logits, y_true=labels, from_logits=True))    
    return loss

## Calculate Gradient

In [35]:
@tf.function
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

## Caculating Model's Accuracy

In [36]:
@tf.function
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

## Optimizer

In [37]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

## Training

In [40]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_dataset:
        grads = grad(model, images, labels)
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(model, images, labels)        
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc), 'train_step: {}'.format(train_step), 'test_step: {}'.format(test_step))
    
print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 1 loss = 0.18647899 train accuracy =  0.9532 test accuracy =  0.9861 train_step: 600 test_step: 100
Epoch: 2 loss = 0.04548335 train accuracy =  0.9903 test accuracy =  0.9886 train_step: 600 test_step: 100
Epoch: 3 loss = 0.03035223 train accuracy =  0.9936 test accuracy =  0.9927 train_step: 600 test_step: 100
Epoch: 4 loss = 0.02266937 train accuracy =  0.9951 test accuracy =  0.9901 train_step: 600 test_step: 100
Epoch: 5 loss = 0.01948784 train accuracy =  0.9965 test accuracy =  0.9905 train_step: 600 test_step: 100
Epoch: 6 loss = 0.01487026 train accuracy =  0.9979 test accuracy =  0.9904 train_step: 600 test_step: 100
Epoch: 7 loss = 0.01374756 train accuracy =  0.9977 test accuracy =  0.9940 train_step: 600 test_step: 100
Epoch: 8 loss = 0.01019295 train accuracy =  0.9983 test accuracy =  0.9921 train_step: 600 test_step: 100
Epoch: 9 loss = 0.00944085 train accuracy =  0.9987 test accuracy =  0.9933 train_step: 600 test_step: 100
